# image generation

In [14]:
# pip install openai

In [8]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [12]:
from openai import OpenAI
client = OpenAI()

response = client.images.generate(
  model="dall-e-2",
  prompt="a white siamese cat",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url

In [13]:
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-wS6XMPfZBTyNBYkzMz47KkiA/user-1AZ7qGPge3H6az6ts5jNvrXU/img-fqkeWkr4corbMgFwrJqoD5vE.png?st=2024-02-24T17%3A17%3A31Z&se=2024-02-24T19%3A17%3A31Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-02-24T10%3A38%3A17Z&ske=2024-02-25T10%3A38%3A17Z&sks=b&skv=2021-08-06&sig=R71twlE/g1cfS4oiM0Jh7y452jPjC6HmHnGIfukw6WY%3D'

# translation

In [17]:
!pip install transformers
# pip install sentencepiece
# pip install sacremoses

In [46]:
from transformers import MarianMTModel, MarianTokenizer

def translate_russian_to_english(text):
    model_name = 'Helsinki-NLP/opus-mt-ru-en'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))
    return tokenizer.decode(translated[0], skip_special_tokens=True)

def translate_english_to_russian(text):
    model_name = 'Helsinki-NLP/opus-mt-en-ru'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))
    return tokenizer.decode(translated[0], skip_special_tokens=True)

In [47]:
russian_text = "Привет, как дела?"
english_text = translate_russian_to_english(russian_text)
back_to_russian = translate_english_to_russian(english_text)

print("Original Russian:", russian_text)
print("Translated to English:", english_text)
print("Back to Russian:", back_to_russian)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Original Russian: Привет, как дела?
Translated to English: Hey, how's it going?
Back to Russian: Привет, как дела?


# deck generation

In [28]:
!pip install langchain

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate, ChatPromptTemplate

system_prompt = "Generate a list of pairs of words related to the given topic. Each pair should contain an English word and its Russian translation, separated by a comma."
user_template = "Now, generate pairs of words on the topic: {topic}."
topic = "zoo"

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")
chat_template = ChatPromptTemplate.from_messages([SystemMessage(content=system_prompt),
                                                  HumanMessagePromptTemplate.from_template(user_template)])
formatted_template = chat_template.format_messages(topic=topic)
message = llm.invoke(formatted_template).content

pairs = message.split('\n')
word_pairs = {pair.split(', ')[0]: pair.split(', ')[1] for pair in pairs if ', ' in pair}

print(word_pairs)


{'zoo': 'зоопарк', 'animal': 'животное', 'keeper': 'смотритель', 'cage': 'клетка', 'elephant': 'слон', 'ticket': 'билет', 'feeding': 'кормление', 'giraffe': 'жираф', 'penguin': 'пингвин', 'enclosure': 'вольер'}


# card generation

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate, ChatPromptTemplate

system_prompt = "Based on the provided list of words, suggest a new word and its Russian translation that is contextually relevant to them."
user_template = "Given these words in the card deck: {words}, suggest a new relevant word and its Russian translation."
existing_words = ["lion", "tiger", "zoo"]
words_string = ', '.join(existing_words)

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")
chat_template = ChatPromptTemplate.from_messages([SystemMessage(content=system_prompt),
                                                  HumanMessagePromptTemplate.from_template(user_template)])
formatted_template = chat_template.format_messages(words=words_string)
message = llm.invoke(formatted_template).content.strip()

suggested_word_pair = message.split('\n')
word_dict = {}

if len(suggested_word_pair) == 2:
    english_word = suggested_word_pair[0].split(': ')[1]
    russian_word = suggested_word_pair[1].split(': ')[1].split(' ')[0]
    word_dict[english_word] = russian_word

print(word_dict)

{'Elephant': 'Слон'}
